In [ ]:
!pip3 install torch
!pip3 install transformers
!pip3 install torchvision
!pip3 install torchaudio

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokeinzer=AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
# Encode and calculate sentiment

In [ ]:
tokens=tokeinzer.encode("I hated this word")

In [ ]:
## Decode
tokeinzer.decode(tokens[0])

tokens=tokeinzer.encode('I hated this, absolutely worst',return_tensors='pt')
result=model(tokens)
result.logits
int(torch.argmax(result.logits))+1

1

In [ ]:
## collect reviews

In [ ]:
url = "https://www.yelp.com/biz/mejico-sydney-2"
headers = {"User-Agent": "Mozilla/5.0"}  # Yelp may block without headers
r = requests.get(url, headers=headers)

soup = BeautifulSoup(r.text, "html.parser")

# Match review text classes (Yelp changes them often, regex helps)
reviews = re.compile("comment")


In [ ]:
reviews[0]


TypeError: 're.Pattern' object is not subscriptable

In [ ]:
## Load reviews into dataframe and score

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.DataFrame(np.array(reviews),columns=["reviews"])

In [ ]:
def sentiment_score(review):
  tokens=tokeinzer.encode(review,return_tensors="pt")
  result=model(tokens)
  return int(torch.argmax(result.logits))+1

In [ ]:
df["sentiment"]=df["review"].apply(lambda x:sentiment_score(x[:512]))

In [ ]:
df

In [ ]:
# https://www.youtube.com/watch?v=szczpgOEdXs&t=560s